In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
sx.frames['1 day'].data


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 269 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 269 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-02,248.05,251.25,244.41,248.00,877576
2024-01-03,247.41,248.48,236.32,238.22,1023215
2024-01-04,239.15,242.70,237.40,238.41,878148
2024-01-05,237.12,240.12,234.90,237.40,792137
2024-01-08,236.00,241.25,234.70,240.35,701465
...,...,...,...,...,...
2024-12-24,435.90,462.78,435.14,462.28,399043
2024-12-26,465.16,465.33,451.02,454.13,508124
2024-12-27,449.37,450.00,426.50,431.66,559640


Error 10148, reqId 607: OrderId 607 that needs to be cancelled cannot be cancelled, state: Cancelled.
Error 10148, reqId 609: OrderId 609 that needs to be cancelled cannot be cancelled, state: Cancelled.
Error 10148, reqId 610: OrderId 610 that needs to be cancelled cannot be cancelled, state: Cancelled.
Error 10148, reqId 612: OrderId 612 that needs to be cancelled cannot be cancelled, state: Cancelled.


In [2]:
import strategies.ta as ta
import strategies.preset_strats as ps
f = sx.frames['1 day']
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [21, 50, 200])
f.plot()
f.data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'HP_hi_10', 'LP_lo_10',
       'HP_hi_3', 'LP_lo_3', 'FFILL_HP_hi_3', 'FFILL_LP_lo_3', 'ATR_50',
       'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper',
       'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2',
       'Sup_2_Upper', 'Sup_2_Lower', 'MA_cl_21', 'MA_cl_50', 'MA_cl_200'],
      dtype='object')

In [3]:
import sb_order
ox = sb_order.OrderX('Strat1', ib, 'TSLA', 'LONG')
ox.set_entry(qty=8, outsideRth=True, limitPrice=400.00)
ox.add_bracket_order(qtyPct=25, stop_price=370.00, target_price=420.00)
ox.add_bracket_order(qtyPct=25, stop_price=350.00, target_price=470.00)
ox.add_stop_order(qtyPct=50, stop_price=395.00)
ox.place_orders(delay_between_orders=1)

Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 6
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 4
Stop quotas:  QtyPct: 50, Qty: 4, Quotas: 0


In [4]:
ox.cancel_orders()

In [3]:
import sb_order
import pandas as pd

    
print(f.data['Res_2'].iat[-1])
print(f.data['Res_1'].iat[-1])
print(f.data['Sup_1'].iat[-1])
print(f.data['Sup_2'].iat[-1])
print(f.data['MA_cl_50'].iat[-1])
print(f.data['FFILL_HP_hi_3'].iat[-1])


oxd = sb_order.OrderXData('Strat1', ib, 'TSLA', 'LONG')
oxd.set_entry(f.data, qty=8, outsideRth=True, limitPrice='close')
oxd.add_bracket_order(f.data, qtyPct=25, stop_price='Sup_1', target_price='FFILL_HP_hi_3')
oxd.add_bracket_order(f.data, qtyPct=25, stop_price='Sup_2', target_price='Res_1')
oxd.add_stop_order(f.data, qtyPct=50, stop_price='Sup_1')
oxd.place_orders(f.data, delay_between_orders=0)
print("Connection status:", ib.isConnected())

nan
488.54
362.8
271.0
346.6318
465.33
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 6
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 4
Stop quotas:  QtyPct: 50, Qty: 4, Quotas: 0
Connection status: True


In [4]:
oxd.modify_stop(f.data, stopNum=1, stop_price='MA_cl_21')
oxd.modify_stop(f.data, stopNum=2, stop_price='MA_cl_50')

Old price: 362.8, New price: 415.68, new > old = True
Old price: 271.0, New price: 346.63, new > old = True


In [4]:
display(oxd.get_orders_as_df())
display(oxd.get_orders_status_as_df())

,parentId,orderId,clientId,permId,action,totalQuantity,orderType,ocaGroup,ocaType,orderRef,lmtPrice,auxPrice,outsideRth,tif,goodTillDate,transmit
0,0,595,12,551174510,BUY,2.0,LMT,,0,Strat1_Entry_1,403.84,0.0,True,,,False
1,595,596,12,551174511,SELL,2.0,STP,,0,Strat1_Stop_1_pct25,0.00,362.8,False,,,False
2,595,597,12,551174512,SELL,2.0,LMT,,0,Strat1_Tget_1_pct25,465.33,0.0,False,,,True
3,0,598,12,551174513,BUY,2.0,LMT,,0,Strat1_Entry_2,403.84,0.0,True,,,False
4,598,599,12,551174514,SELL,2.0,STP,,0,Strat1_Stop_2_pct25,0.00,271.0,False,,,False
5,598,600,12,551174515,SELL,2.0,LMT,,0,Strat1_Tget_2_pct25,488.54,0.0,False,,,True
6,0,601,12,551174516,BUY,4.0,LMT,,0,Strat1_Entry_3,403.84,0.0,True,,,False
7,601,602,12,551174517,SELL,4.0,STP,,0,Strat1_Stop_3_pct50,0.00,362.8,False,,,True


,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld,mktCapPrice
0,595,Submitted,0.0,2.0,0.0,551174510,0,0.0,12,,0.0
1,596,PreSubmitted,0.0,2.0,0.0,551174511,595,0.0,12,"child,trigger",0.0
2,597,PreSubmitted,0.0,2.0,0.0,551174512,595,0.0,12,child,0.0
3,598,Submitted,0.0,2.0,0.0,551174513,0,0.0,12,,0.0
4,599,PreSubmitted,0.0,2.0,0.0,551174514,598,0.0,12,"child,trigger",0.0
5,600,PreSubmitted,0.0,2.0,0.0,551174515,598,0.0,12,child,0.0
6,601,Submitted,0.0,4.0,0.0,551174516,0,0.0,12,,0.0
7,602,PreSubmitted,0.0,4.0,0.0,551174517,601,0.0,12,"child,trigger",0.0


In [5]:
oxd.cancel_orders()